In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Async

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

config = {"configurable": {"thread_id": "abc123"}}


In [7]:
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
aapp = workflow.compile(checkpointer=MemorySaver())



In [8]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Hello Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [9]:

query = "What's my name?"
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! You told me that when we started talking.


# Adding a System Message

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrrr, greetings to ye, Bob me hearty! 'Tis a grand day to be havin' a chat, don't ye think? What be bringin' ye to these fair waters? Are ye lookin' fer treasure, or just passin' the time with a swashbucklin' pirate like meself?


# Trimming

In [8]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

In [9]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages


trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import aiosqlite


db_path = 'checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)

#memory = SqliteSaver(conn)
# Does not work
#memory = AsyncSqliteSaver(conn)


memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
        
print(type(app))

<class 'langgraph.graph.state.CompiledStateGraph'>


In [3]:
from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="What is 1+1?")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)
    async for event in graph.astream_events(inputs, config, version="v1"):
        pprint(event)

/Users/dan/miniforge3/envs/deep-dive/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={})]}},
 'event': 'on_chain_start',
 'metadata': {'thread_id': 'abc12'},
 'name': 'LangGraph',
 'parent_ids': [],
 'run_id': '5df10a42-1d4a-4db7-b2c8-338fb650fda9',
 'tags': []}
{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={})]}},
 'event': 'on_chain_start',
 'metadata': {'langgraph_checkpoint_ns': '__start__:667b0be5-b56a-7127-d177-5694f2ee96b4',
              'langgraph_node': '__start__',
              'langgraph_path': ('__pregel_pull', '__start__'),
              'langgraph_step': 0,
              'langgraph_triggers': ['__start__'],
              'thread_id': 'abc12'},
 'name': '__start__',
 'parent_ids': [],
 'run_id': '3e8d95f3-b3c2-45d8-b200-8603b14d0498',
 'tags': ['graph:step:0', 'langsmith:hidden']}
{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_met

NotImplementedError in LogStreamCallbackHandler.on_llm_end callback: NotImplementedError('Trying to load an object that doesn\'t implement serialization: {\'lc\': 1, \'type\': \'not_implemented\', \'id\': [\'ollama\', \'_types\', \'Message\'], \'repr\': "Message(role=\'assistant\', content=\'\', images=None, tool_calls=None)"}')


{'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-02-21T13:14:27.712454Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18460429459, 'load_duration': 571318209, 'prompt_eval_count': 32, 'prompt_eval_duration': 16467000000, 'eval_count': 13, 'eval_duration': 1199000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a8b43770-f131-4493-b709-09a0d10b39e2', usage_metadata={'input_tokens': 32, 'output_tokens': 13, 'total_tokens': 45})},
 'event': 'on_chat_model_stream',
 'metadata': {'checkpoint_ns': 'model:457b7390-71d6-349b-120f-e1b57096a593',
              'langgraph_checkpoint_ns': 'model:457b7390-71d6-349b-120f-e1b57096a593',
              'langgraph_node': 'model',
              'langgraph_path': ('__pregel_pull', 'model'),
              'langgraph_step': 1,
              'langgraph_triggers': ['start:model'],
              'ls_model_name': 'llama3.3',
    

In [6]:
# THIS COULD BE THE ONE I NEED

from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="Hi! I'm Bob.")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)        
    async for chunk, metadata in graph.astream(
        inputs,
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):  # Filter to just model responses
            print(chunk.content, end="")

Nice to meet you, Bob! It's great to have a name to go along with our conversation. How's your day going so far, Bob?

In [38]:
import json


config = {"configurable": {"thread_id":  "abc12"}}



with SqliteSaver.from_conn_string(db_path) as checkpointer:
    graph = workflow.compile(checkpointer=checkpointer)
    state_history = graph.get_state_history(config) 
    values = next(state_history).values  
    print(values) 
    print(len(values['messages']))
    for message in values['messages']:
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        else:
            print(f"AI:    {message.content}")
    
    #state_history_list = list(state_history)
    #print(state_history_list)
    # for state in state_history:
    #     print(state)
    #     # print(f"Step: {state.step}")
    #     # print(f"Created At: {state.created_at}")
    #     # print(f"Values: {state.values}")
    #     # print("-----")
 
    #list(graph.get_state_history(config))

{'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={}, id='798105e7-0764-4948-bd73-7b1a1a3a6382'), AIMessage(content='The answer to 1 + 1 is 2.', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-02-21T13:14:27.712454Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18460429459, 'load_duration': 571318209, 'prompt_eval_count': 32, 'prompt_eval_duration': 16467000000, 'eval_count': 13, 'eval_duration': 1199000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-a8b43770-f131-4493-b709-09a0d10b39e2', usage_metadata={'input_tokens': 32, 'output_tokens': 13, 'total_tokens': 45}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='46cb3236-78b7-4dd8-a2c9-c17a22e91137'), AIMessage(content="I don't have any information about your name. Our conversation just started, and you haven't shared that with me yet! Would you like to tell m

In [ ]:
query = "What is my name"
input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

In [ ]:
import asyncio

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph

builder = StateGraph(int)
builder.add_node("add_one", lambda x: x + 1)
builder.set_entry_point("add_one")
builder.set_finish_point("add_one")
async with AsyncSqliteSaver.from_conn_string("checkpoints.db") as memory:
    graph = builder.compile(checkpointer=memory)
    coro = graph.ainvoke(1, {"configurable": {"thread_id": "thread-1"}})
    print(asyncio.run(coro))

# Async history

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import START, MessagesState, StateGraph



system_prompt = "Answer all questions to the best of your ability. Answer concisely but correctly. If you do not know the answer, just say 'I don’t know.'"
model = init_chat_model("deepseek-r1:32b", model_provider="ollama")

trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

graph = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}

graph.add_edge(START, "model")
graph.add_node("model", call_model)



In [3]:
print("IN get_chat_interactions_count")
interactions_count = 0
db_path="/Users/dan/Code/LLM/DeepDive/backend/resources/checkpoints.db"
chat_id="ea9123ff-9a8e-46c0-a53f-22b8f88e3202"

with SqliteSaver.from_conn_string(db_path) as checkpointer:            
    config = {"configurable": {"thread_id": chat_id}}
    print("before compiled_graph")
    compiled_graph = graph.compile(checkpointer=checkpointer)
    print("after compiled_graph")
    state_history = compiled_graph.get_state_history(config) 
    print("after state_history")
    last_interaction = next(state_history, None)
    print("after last_interaction")
    if last_interaction:
        values = last_interaction.values  
        if 'messages' in values:
            interactions_count = len(values['messages'])
            
interactions_count

IN get_chat_interactions_count
before compiled_graph
after compiled_graph
after state_history
after last_interaction


6

In [7]:
import sys
import os
sys.path.append('/Users/dan/Code/LLM/DeepDive/backend')

import app.app_session
file_path = app.app_session.__file__
parent_directory = os.path.dirname(file_path)
grandparent_directory = os.path.dirname(parent_directory)
grandparent_directory

'/Users/dan/Code/LLM/DeepDive/backend'